# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [189]:
import pandas as pd
import requests

# 1) seu código aqui
df_sus = pd.read_csv('SINASC_RO_2019.csv')
df_sus.shape

(27028, 69)

In [190]:
df_sus = df_sus.drop_duplicates()
df_sus.shape


(27028, 69)

In [191]:
# 2) seu código aqui
df_sus.isnull().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [192]:
# 3) seu código aqui
df_sus = df_sus[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
df_sus

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [193]:
df_sus.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [194]:
# 4) seu código aqui
df_sus = df_sus[~df_sus['APGAR5'].isnull()]



In [195]:
df_sus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26925 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     26925 non-null  int64  
 1   IDADEMAE    26925 non-null  int64  
 2   ESTCIVMAE   26610 non-null  float64
 3   ESCMAE      26615 non-null  object 
 4   QTDFILVIVO  25359 non-null  float64
 5   GESTACAO    25709 non-null  object 
 6   GRAVIDEZ    26849 non-null  object 
 7   CONSULTAS   26925 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 2.1+ MB


In [196]:
df_sus.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [197]:
# 5) seu código aqui
df_sus[['ESTCIVMAE' , 'CONSULTAS']].fillna(9 , inplace=True)


C:\Users\g_abi\AppData\Local\Temp\ipykernel_11640\3925723555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus[['ESTCIVMAE' , 'CONSULTAS']].fillna(9 , inplace=True)


In [198]:
df_sus.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [199]:
# 6) Seu código aqui
df_sus['QTDFILVIVO'].fillna(0 , inplace = True)


C:\Users\g_abi\AppData\Local\Temp\ipykernel_11640\3420623303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus['QTDFILVIVO'].fillna(0 , inplace = True)


In [200]:
# 7) seu código aqui
df_sus['ESCMAE'].fillna(9 , inplace = True) # Não consegui ver um coorelação entre a escolaridade da mãe com o resultado final da análise. Utilizei o '9' por conta do código na planilha.
df_sus['ESTCIVMAE'].fillna(9 , inplace = True)# Não consegui ver um coorelação entre o estado cívil da mãe com o resultado final da análise. Utilizei o '9' por conta do código na planilha.

C:\Users\g_abi\AppData\Local\Temp\ipykernel_11640\2541565369.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus['ESCMAE'].fillna(9 , inplace = True) # Não consegui ver um coorelação entre a escolaridade da mãe com o resultado final da análise. Utilizei o '9' por conta do código na planilha.
C:\Users\g_abi\AppData\Local\Temp\ipykernel_11640\2541565369.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus['ESTCIVMAE'].fillna(9 , inplace = True)# Não consegui ver um coorelação entre o estado cívil da mãe com o resultado final da análise. Utilizei o '9' por conta do código na planilha.


In [201]:
qtd_tipo_gravidez = ((df_sus['GRAVIDEZ'].value_counts().sum()) - 
 df_sus[df_sus['GRAVIDEZ'] == 9].value_counts().sum()) #calculando o total do tipo gravidez retirando os valores 'ignorados'

In [202]:
(df_sus['GRAVIDEZ'].value_counts() / qtd_tipo_gravidez) * 100

Única              98.163805
Dupla               1.817572
Tríplice e mais     0.018623
Name: GRAVIDEZ, dtype: float64

In [203]:
df_sus['GRAVIDEZ'].fillna('Única' , inplace = True) # A gravidez única representa 98.44% dos tipos de gravidez no estudo, como a quantidade de NaN representava uma parcela muito inferior , substituí por uma gravidez do tipo única. 

C:\Users\g_abi\AppData\Local\Temp\ipykernel_11640\3144551992.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus['GRAVIDEZ'].fillna('Única' , inplace = True) # A gravidez única representa 98.44% dos tipos de gravidez no estudo, como a quantidade de NaN representava uma parcela muito inferior , substituí por uma gravidez do tipo única.


In [204]:
df_sus['GESTACAO'].value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [205]:
gestacao_faltantes = df_sus['GESTACAO'].isnull().sum()

In [206]:
gestacao_total = (df_sus['GESTACAO'].value_counts().sum()) - gestacao_faltantes

In [207]:
(gestacao_faltantes / gestacao_total) * 100

4.964683787204508

In [208]:
(df_sus['GESTACAO'].value_counts() / gestacao_total) * 100

37 a 41 semanas        91.956886
32 a 36 semanas         9.120973
42 semanas e mais       2.849794
28 a 31 semanas         0.661413
22 a 27 semanas         0.347038
Menos de 22 semanas     0.028580
Name: GESTACAO, dtype: float64

In [209]:
df_sus['GESTACAO'].fillna('37 a 41 semanas' , inplace = True) # A gestação de 37 a 41 semanas representa 91.95% dos tipos de gestação no estudo, como a quantidade de NaN representava uma parcela muito inferior , substituí pela gestação com maior porcentagem. 

C:\Users\g_abi\AppData\Local\Temp\ipykernel_11640\3518255346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus['GESTACAO'].fillna('37 a 41 semanas' , inplace = True) # A gestação de 37 a 41 semanas representa 91.95% dos tipos de gestação no estudo, como a quantidade de NaN representava uma parcela muito inferior , substituí pela gestação com maior porcentagem.


In [210]:
df_sus.isnull().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [211]:
# 8) seu código aqui
df_sus

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [212]:
df_sus.loc[(df_sus['APGAR5']>=8) & (df_sus['APGAR5']<=10) , 'classificacao'] = 'Normal'
df_sus.loc[(df_sus['APGAR5']>=6) & (df_sus['APGAR5']<=7) , 'classificacao'] = 'asfixia leve'
df_sus.loc[(df_sus['APGAR5']>=4) & (df_sus['APGAR5']<=5) , 'classificacao'] = 'asfixia moderada'
df_sus.loc[(df_sus['APGAR5']==0) & (df_sus['APGAR5']<=3) , 'classificacao'] = 'asfixia severa'

C:\Users\g_abi\AppData\Local\Temp\ipykernel_11640\3211806285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus.loc[(df_sus['APGAR5']>=8) & (df_sus['APGAR5']<=10) , 'classificacao'] = 'Normal'


In [213]:
df_sus

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,classificacao
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal


In [214]:
(df_sus['classificacao'].value_counts() / df_sus['classificacao'].value_counts().sum()) * 100

Normal              98.492631
asfixia leve         1.191008
asfixia moderada     0.253089
asfixia severa       0.063272
Name: classificacao, dtype: float64

In [215]:
df_sus

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,classificacao
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal


In [218]:
# 9) seu código aqui
df_sus = df_sus.rename(columns = {'LOCNASC' : 'loc_nasc' ,
                         'IDADEMAE' : 'idade_mae' ,
                         'ESTCIVMAE' : 'est_civ_mae' ,
                         'ESCMAE' : 'esc_mae' ,
                         'QTDFILVIVO' : 'qtd_fil_vivo' ,
                        'GESTACAO' : 'gestacao' ,
                        'GRAVIDEZ' : 'gravidez' ,
                        'CONSULTAS' : 'consultas' ,
                        'APGAR5' : 'apgar_5'
                                 })

In [219]:
df_sus

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,classificacao
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal
